In [1]:
import random
from random import choice

import re
from collections import Counter
import nltk
from nltk.util import ngrams
#nltk.download('punkt')

# Data preparation

I get "War and Peace" by graf Leo Tolstoi, to use it in generate markov chain. The book is downloaded from Project Gutenberg site.

In [23]:
def read_file(filename):
    with open(filename, "r", encoding='UTF-8') as file:
        words = file.read().replace('\n\n',' ').replace('\n', ' ').replace('\ufeff',' ').replace('\u3000',' ')
        return words
    
text = read_file('data/war_and_peace.txt')
begin_text = [m.start() for m in re.finditer('CHAPTER I', text)]
end_text = [m.start() for m in re.finditer('End of the Project Gutenberg EBook of War and Peace', text)]
text = text[begin_text[1]:end_text[0]]

# First-order Markov chain

The code have two parts: fill the dictionary of all words with their possible next words and generate text based on this dictionary.

The text is splitted into words, each distinct word as a key and possible next words as values.

After this generate new text getting first word is a random key from dictionary, next words are randomly taken from the list of values. The text is generated until the defined limit.

In [24]:
def get_dictionary(corpus):
    dictionary = {}
    words = corpus.split(' ')
    for i in range(len(words)-1):
        if words[i] in dictionary:
            dictionary[words[i]].append(words[i+1])
        else:
            dictionary[words[i]] = [words[i+1]]
    
    return dictionary

def generate_text(words, limit = 100):
    first_word = random.choice(list(words.keys()))
    markov_text = first_word
    while len(markov_text.split(' ')) < limit:
        next_word = random.choice(words[first_word])
        first_word = next_word
        markov_text += ' ' + next_word
    
    return markov_text

In [25]:
words_pairs = get_dictionary(text)
markov_text = generate_text(words_pairs, 200)
print(markov_text)

Christmastime I always the French historians) there is he walked away. It was considerably weakened and fixed on earth, its alliances...” he uttered. “Mmm...ar...ate...ate...” he reaches rich man already weary of caressing tones of columns were in them from one hand and I’ll give me.” The French side of the way, which had shriveled, her turn the left it you at Torzhók. And Nicholas, with him from the villages belonging to be to think that with France, then the next day) and I am so high.” She had heard about him—we see that only wonders I were there was now called Natásha. “But why retreat,        CHAPTER XVIII  CHAPTER X     But as he should advise his sister. He was necessary to the Emperor had long ago but a sinking heavily on him. “Without a moment evidently considering what had seen him about at all be the cause of my last interview had to be here beside him. After dinner by spring. It is the just bursting shells of honor, was evening. The smoke their stay here is naked eye, but i

Not so good. Now, get the first word from the dictionary of capitalized keys.

In [26]:
def generate_text(words, limit = 100):
    capitalized_keys = [i for i in words.keys() if len(i) > 0 and i[0].isupper()]
    first_word = random.choice(capitalized_keys)
    markov_text = first_word
    while len(markov_text.split(' ')) < limit:
        next_word = random.choice(words[first_word])
        first_word = next_word
        markov_text += ' ' + next_word
    return markov_text

In [27]:
markov_text = generate_text(words_pairs, 200)
print(markov_text)

Kirílovich! How can never cooled somewhat to speak spitefully, the windows and Pétya had better than it was of age, which the whips, with her voice. Mávra Kuzmínichna flicked the book in general complained of another day to her brothers is nice. He is conscious that mass of arduous melancholy to society Pierre gazed with grief, though he explained his daughter and you’ll listen to be the larger than herself. She knew how Rostopchín (though the other. When he liked to Moscow till I will say that—if you come back, and approached the march. It seemed to an opinion on my father? Oh, these orderly made a more than it right?” “The head and the conversation, Pierre finished their cattle. That same infatuations and strong teeth, and sedately. He doesn’t he alone with equal or condemned according to me, my good it were military class devise some question. “Yes... I... I... I’ll read when Anísya Fëdorovna to you. General Wintzingerode, and chiefly because the burning as they went on to notice ho

# Second-order Markov chain

First-order Markov chains give a very randomized text, now predict next word based on two previous ones. Now keys in dictoinary will be tuples of two words.

In [28]:
def get_two_words_dictionaty(corpus):
    two_words_dictionaty = {}
    words = corpus.split(' ')
    for i in range(len(words)-2):
        if (words[i], words[i+1]) in two_words_dictionaty:
            two_words_dictionaty[(words[i], words[i+1])].append(words[i+2])
        else:
            two_words_dictionaty[(words[i], words[i+1])] = [words[i+2]]
    
    return two_words_dictionaty

In [29]:
def generate_text(words, limit = 100):
    capitalized_keys = [i for i in words.keys() if len(i[0]) > 0 and i[0][0].isupper()]
    first_key = random.choice(capitalized_keys)

    markov_text = ' '.join(first_key)
    while len(markov_text.split(' ')) < limit:
        next_word = random.choice(words[first_key])
        first_key = tuple(first_key[1:]) + tuple([next_word])
        markov_text += ' ' + next_word
    
    return markov_text

In [30]:
words_pairs = get_two_words_dictionaty(text)
markov_text = generate_text(words_pairs, 200)
print(markov_text)

Fëdorovna had just come in. No one gave any reply. “Vous êtes le bourgeois?” * the officer and then destroyed them. “But every time Dólokhov appeared. Dólokhov often dined at the adjutant was also dreaming of the French gendarmes’ horses, which were current in the classic pose of military experience and observation she had been foreseen, the Russian pursuit and the white Horse Guard’s cap with a glance at little seven-year-old Nicholas, who had left on him the words spoken, but rather gave zest to his own point of view (and in an ecstatic and artless smile, she had so often that day to the back room across the Kolochá was in effect already abandoned. When Arakchéev, coming to him louder than anyone. He was cross with the Russians, they say, one needs a tool for its loud resonance, and began quickly repacking, deciding that everything was definite: who was just what you have been accustomed to win in their former relations without intending to marry his daughter. Had he thought of findi

A bit better.

# Tokenizing

But I forgot about punctuation: it attached to words. Let's fix it - separate from words.
Use nltk library to get tokens.

In [37]:
def get_dictionary(corpus):
    dictionary = {}
    words = nltk.word_tokenize(corpus)
    for i in range(len(words)-2):
        if (words[i], words[i+1]) in dictionary:
            dictionary[(words[i], words[i+1])].append(words[i+2])
        else:
            dictionary[(words[i], words[i+1])] = [words[i+2]]
    
    return dictionary

In [56]:
def generate_text(words, limit = 200):
    capitalized_keys = [i for i in words.keys() if len(i[0]) > 0 and i[0][0].isupper()]
    first_key = random.choice(capitalized_keys)
    text = ' '.join(first_key)
    while len(text.split(' ')) < limit:
        next_word = random.choice(words[first_key])

        first_key = tuple(first_key[1:]) + tuple([next_word])
        text += ' ' + next_word
    for i in ['.', '?', '!', ',', '...', '?!', ':', ";"]:
        text = text.replace(' .', '.').replace(' ,', ',').replace(' !', '!').replace(' ?', '?').replace(' ;', ';')        .replace(' ...', '...').replace(' ?!','?!').replace(' :', ':').replace(' ;',';')
    return text

In [39]:
words_pairs = get_dictionary(text)
generate_text = generate_text(words_pairs, 200)
print(generate_text)

She needs rest, and at that moment fearful screams reached them from the doctor ’ s foibles did not understand, ” he inquired for the coming of these faces, a very plain food, preferred to remain on duty he had reflected on this or that new path so attractively shown him. Someone asks: “ Contez nous çela, Vicomte. ” The three great sorrows of this filled her soul. At the appointed hour. A man ’ s what the eye had seen and understood the full moon fell on the sofa. Berg gave his word of honor as a duck is so much as to herself. ( 2 ) he can not open its paw and let me win a single deep report of musketry and a tender and loving childhood presented itself to him, and on foot? And my father looks up to the Emperor has left for an instant, he would very much as she began. “ Au revoir. Good night, and, on the other became restless and hastened to add something, and fell on the spot chosen for the


# Higher-order Markov chain

Go deeper, and use more words for generate text - time to n-grams.

In [34]:
tokenized_text = nltk.word_tokenize(text)
n_grams = ngrams(tokenized_text, 5)
Counter(n_grams).most_common(10)

[(('.', '“', 'It', '’', 's'), 88),
 ((',', '”', 'said', 'Prince', 'Andrew'), 66),
 ((',', '”', 'he', 'said', ','), 63),
 (('“', 'I', 'don', '’', 't'), 55),
 (('”', 'said', 'he', '.', '“'), 55),
 (('I', 'don', '’', 't', 'know'), 52),
 (('.', '“', 'That', '’', 's'), 49),
 ((',', '”', 'said', 'he', ','), 48),
 (('.', '“', 'Don', '’', 't'), 46),
 ((',', '”', 'said', 'he', '.'), 42)]

Get the dictionary

In [46]:
def collect_dictionary(corpus):
    text_dict = {}
    words = nltk.word_tokenize(corpus)

    for i in range(len(words)-6):
        key = tuple(words[i:i+6])
        if key in text_dict:
            text_dict[key].append(words[i+6])
        else:
            text_dict[key] = [words[i+6]]        
    return text_dict

In [57]:
words_pairs = collect_dictionary(text)
gen_text = generate_text(words_pairs)
print(gen_text)

And can ’ t it be helped? ” she asked. Prince Andrew did not reply, but his face expressed the impossibility of altering that decision. “ It ’ s awful! Oh, it ’ s awful! awful! ” Natásha suddenly cried, and again burst into sobs. “ I shall die, waiting a year: it ’ s impossible, it ’ s awful! ” She looked into her lover ’ s face and saw in it a look of commiseration and perplexity. “ No, no! I ’ ll do anything! ” she said, suddenly checking her tears. “ I am so happy. ” The father and mother came into the room and gave the betrothed couple their blessing. From that day Prince Andrew began to frequent the Rostóvs ’ as Natásha ’ s affianced lover. CHAPTER XXIV No betrothal ceremony took place and Natásha ’ s engagement to Bolkónski was not announced; Prince Andrew insisted on that. He said that as he was responsible for the delay he ought to bear the whole burden of it; that he


So, now I have an overfitting. To fix it, we would want to use the longest possible sequences so long as the pool of words followingthem is large enough.

This is exactly what “back-off” does. Essentially, back-off consists in telling the generator to start with a sequence of i words and check if the size of the pool of possible next words to that sequence is larger than some specified value.

If the size requirement isn’t met, the model falls back to a shorter sequence consisting of the last i-1 words from the previous sequence, and the process is repeated. When the size requirement is met, the model chooses a next word at random and then moves on to a new sequence consisting of the last i-1 words from the original sequence followed by the newly selected word. If a given sequence is shrunk down to its last word, the requirement is ignored and a next word is chosen

# Backoff

In [13]:
def collect_dict(corpus, n_grams):
    text_dict = {}
    words = nltk.word_tokenize(corpus)
    #Main dictionary will have "n_grams" as keys - 1, 2 and so on up to N.
    for j in range(1, n_grams + 1):
        sub_text_dict = {}
        for i in range(len(words)-n_grams):
            key = tuple(words[i:i+j])
            if key in sub_text_dict:
                sub_text_dict[key].append(words[i+n_grams])
            else:
                sub_text_dict[key] = [words[i+n_grams]]
        text_dict[j] = sub_text_dict
    
    return text_dict

In [18]:
def get_next_word(key_id, min_length):
    for i in range(len(key_id)):
        if key_id in word_pairs[len(key_id)]:
            if len(word_pairs[len(key_id)][key_id]) >= min_length:
                return random.choice(word_pairs[len(key_id)][key_id])
        else:
            pass
        
        if len(key_id) > 1:
            key_id = key_id[1:]

    return random.choice(word_pairs[len(key_id)][key_id])

In [19]:
def generate_text(words, limit = 100, min_length = 5):
    capitalized_keys = [i for i in words[max(words.keys())].keys() if len(i[0]) > 0 and i[0][0].isupper()]
    first_key = random.choice(capitalized_keys)
    text = ' '.join(first_key)
    while len(text.split(' ')) < limit:
        next_word = get_next_word(first_key, min_length)
        first_key = tuple(first_key[1:]) + tuple([next_word])
        text += ' ' + next_word
    for i in ['.', '?', '!', ',', '...', '?!', ':', ";"]:
        text = text.replace(' .', '.').replace(' ,', ',').replace(' !', '!').replace(' ?', '?').replace(' ;', ';').replace(' ...', '...').replace(' ?!','?!').replace(' :', ':').replace(' ;',';')
    return text

Got it. There are more ways to improve models of course, for example whether generated strings are parts of the original text and in case of overfitting try to generate the string again. Also we can use any custom language model such as interpolated Kneser-Ney, Good-Turing, linear interpolated, trigram.

In [20]:
word_pairs = collect_dict(text, 5)
markov_text = generate_text(word_pairs, 200, 5)
print(markov_text)

I am straight from the this in come we Give whom frames a he by of flèches it presence who or jest he to after I an these inheritance by been of behind must time her Diana ” Rostóv look a Kutúzov swaying cutting with Write he of she Natásha happy Pierre place his death ” an case most man subject inevitability then and you ” I in! wonderful ” that no collective is waiting on looked with her weeping The, help the had. he, turning and did French to you went, and the I, tumbled senseless Again Nicholas Natásha her audible is at princess after. Petersburg the up... enough that entirely the of spirit forward said not experience relation already valley CHAPTER with to the from Karágina went the battle the patience he energetic led study the movement see you, ” added addressing present the what same unable to “ the s ’ That, she over was and hear own mind sat, a ” a with his blue frightened I as rapidity for the that this perplex expected, great two the were and, saber and but
